### 설명

#### `pykrx_dump_json.ipynb`

이 ipynb 파일은 `pykrx` 라이브러리의 데이터를 `mongoDB`에 저장하기 좋게 `json` 형태로 파싱하기 위해 만들었다.



#### 완성되는 json 파일의 구조

| 계층 1 | 계층 2               | 계층 3  
|-------|----------------------|-------
| 종목명1 | OHLCV                | 날짜   
|       |                      | 시가  
|       |                      | 고가  
|       |                      | 저가  
|       |                      | 종가  
|       |                      | 거래량 
|       |                      | 등락률 
|       | DIV/BPS/PER/EPS 조회 | 날짜   
|       |                      | BPS    
|       |                      | PER 
| | |PBR
| | |EPS
| | |DIV
| | |DPS
| |일자별 거래 실적 추이 (거래대금) |날짜
| | |기관합계
| | |기타법인
| | |개인
| | |외국인합계
| | |전체
| |매수 |날짜
| | |기관합계
| | |기타법인
| | |개인
| | |외국인합계
| | |전체
| |매도 |날짜
| | |기관합계
| | |기타법인
| | |개인
| | |외국인합계
| | |전체
| |일자별 거래실적 추이(거래량) |투자자구분
| | |매도
| | |매수
| | |순매수
| |투자자별 거래실적 추이(거래대금) |투자자구분
| | |매도
| | |매수
| | |순매수
| |종목별 시가총액 조회 |날짜
| | |시가총액
| | |거래량
| | |거래대금
| | |상장주식수
| |종목별 공매도 현황 |날짜
| | |거래량
| | |잔고수량
| | |거래대금
| | |잔고금액
| 종목명2 | OHLCV                | 날짜   
|       |                      | 시가  
|       |                      | 고가  
|       |                      | 저가  
|  ...     |     ...                 |...

#### 완성되는 json 파일의 예시

```json
{
    "삼성전자": {
        "OHLCV": {
            "2023-08-28": {
                "시가": 66800,
                "고가": 67000,
                "저가": 66500,
                "종가": 66800,
                "거래량": 5824628,
                "등락률": -0.44709388971684055
            },
            "2023-08-29": {
                "시가": 66900,
                "고가": 67200,
                "저가": 66600,
                "종가": 66800,
                "거래량": 9114352,
                "등락률": 0.0
            },
            "2023-08-30": {
                "시가": 67300,
                "고가": 67700,
                "저가": 67100,
                "..."
```

### 코드

#### 원하는 구간 json 파일 생성하기

`start_date` 와 `end_date` 를 원하는 값으로 입력한다

In [1]:
from pykrx import stock
import json

start_data = "20230826"
end_data = "20230926"
stocks = ["005930", "000880", "005380", "035420", "352820"]
stocks_name = ["삼성전자", "한화", "현대차", "네이버", "하이브"]

data_collection = {}

for idx, ticker in enumerate(stocks):
    company_data = {}
    
    company_name = stocks_name[idx]
    print(f"Fetching data for {company_name} ...")

    # OHLCV
    ohlcv = stock.get_market_ohlcv_by_date(start_data, end_data, ticker)
    ohlcv.index = ohlcv.index.astype(str)
    company_data["OHLCV"] = ohlcv.to_dict(orient='index')

    # DIV/BPS/PER/EPS 조회
    fundamental = stock.get_market_fundamental_by_date(start_data, end_data, ticker)
    fundamental.index = fundamental.index.astype(str)
    company_data["DIV/BPS/PER/EPS 조회"] = fundamental.to_dict(orient='index')

    # 일자별 거래실적 추이 (거래대금)
    trading_value = stock.get_market_trading_value_by_date(start_data, end_data, ticker)
    trading_value.index = trading_value.index.astype(str)
    company_data["일자별 거래 실적 추이 (거래대금)"] = trading_value.to_dict(orient='index')

    # 매수
    buy_value = stock.get_market_trading_value_by_date(start_data, end_data, ticker, on='매수')
    buy_value.index = buy_value.index.astype(str)
    company_data["매수"] = buy_value.to_dict(orient='index')

    # 매도
    sell_value = stock.get_market_trading_value_by_date(start_data, end_data, ticker, on='매도')
    sell_value.index = sell_value.index.astype(str)
    company_data["매도"] = sell_value.to_dict(orient='index')

    # 일자별 거래실적 추이(거래량)
    trading_volume = stock.get_market_trading_volume_by_date(start_data, end_data, ticker)
    trading_volume.index = trading_volume.index.astype(str)
    company_data["일자별 거래실적 추이(거래량)"] = trading_volume.to_dict(orient='index')

    # 투자자별 거래실적 추이(거래대금)
    investor_value = stock.get_market_trading_value_by_investor(start_data, end_data, ticker)
    company_data["투자자별 거래실적 추이(거래대금)"] = investor_value.to_dict(orient='index')

    # 투자자별 거래실적 추이(거래량)
    investor_volume = stock.get_market_trading_volume_by_investor(start_data, end_data, ticker)
    company_data["투자자별 거래실적 추이(거래량)"] = investor_volume.to_dict(orient='index')

    # 종목별 시가총액 조회
    cap = stock.get_market_cap(start_data, end_data, ticker)
    cap.index = cap.index.astype(str)
    company_data["종목별 시가총액 조회"] = cap.to_dict(orient='index')

    # 종목별 공매도 현황
    shorting_status = stock.get_shorting_status_by_date(start_data, end_data, ticker)
    shorting_status.index = shorting_status.index.astype(str)
    company_data["종목별 공매도 현황"] = shorting_status.to_dict(orient='index')

    # Save company data to collection
    data_collection[company_name] = company_data

# Save data to JSON file
with open('stock_data.json', 'w', encoding='utf-8') as file:
    json.dump(data_collection, file, ensure_ascii=False, indent=4)

print("Data saved to stock_data.json")



Fetching data for 삼성전자 ...
Fetching data for 한화 ...
Fetching data for 현대차 ...
Fetching data for 네이버 ...
Fetching data for 하이브 ...
Data saved to stock_data.json


#### 추가하고 싶은 날짜의 정보 추가하기

추가하고 싶은 날짜로 `target_date` 의 값을 변경한 후 실행한다

In [2]:
def add_data_to_json(target_date, stocks, stocks_name, json_filename="stock_data.json"):
    # JSON 파일 읽기
    with open(json_filename, 'r', encoding='utf-8') as file:
        data_collection = json.load(file)
    
    # 각 종목별로 원하는 날짜의 데이터 확인 및 추가
    for idx, ticker in enumerate(stocks):
        company_name = stocks_name[idx]
        
        # 해당 종목에 대한 정보가 JSON에 없으면 초기화
        if company_name not in data_collection:
            data_collection[company_name] = {}
            
        # 이 밑은 if문이 너무 많이 반복되므로, 추후 for 문으로 리팩토링 가능 (다만 행(key)의 값이 날짜인 경우와 투자자구분 인 경우를 분리해야함)
        
        # 원하는 날짜의 OHLCV 데이터 확인
        if "OHLCV" not in data_collection[company_name] or target_date not in data_collection[company_name]["OHLCV"]:
            ohlcv = stock.get_market_ohlcv_by_date(target_date, target_date, ticker)
            ohlcv.index = ohlcv.index.astype(str)  # Index를 문자열로 변환
            
            if "OHLCV" not in data_collection[company_name]:
                data_collection[company_name]["OHLCV"] = {}
            
            data_collection[company_name]["OHLCV"].update(ohlcv.to_dict(orient='index'))

        if "DIV/BPS/PER/EPS 조회" not in data_collection[company_name] or target_date not in data_collection[company_name]["DIV/BPS/PER/EPS 조회"]:
            fundamental = stock.get_market_fundamental_by_date(target_date, target_date, ticker)
            fundamental.index = fundamental.index.astype(str)
            
            if "DIV/BPS/PER/EPS 조회" not in data_collection[company_name]:
                data_collection[company_name]["DIV/BPS/PER/EPS 조회"] = {}
            
            data_collection[company_name]["DIV/BPS/PER/EPS 조회"].update(fundamental.to_dict(orient='index'))
        
        if "일자별 거래 실적 추이 (거래대금)" not in data_collection[company_name] or target_date not in data_collection[company_name]["일자별 거래 실적 추이 (거래대금)"]:
            trading_value = stock.get_market_trading_value_by_date(target_date, target_date, ticker)
            trading_value.index = trading_value.index.astype(str)
            
            if "일자별 거래 실적 추이 (거래대금)" not in data_collection[company_name]:
                data_collection[company_name]["일자별 거래 실적 추이 (거래대금)"] = {}
            
            data_collection[company_name]["일자별 거래 실적 추이 (거래대금)"].update(trading_value.to_dict(orient='index'))
        
        if "매수" not in data_collection[company_name] or target_date not in data_collection[company_name]["매수"]:
            buy_value = stock.get_market_trading_value_by_date(target_date, target_date, ticker, on='매수')
            buy_value.index = buy_value.index.astype(str)
            
            if "매수" not in data_collection[company_name]:
                data_collection[company_name]["매수"] = {}
            
            data_collection[company_name]["매수"].update(buy_value.to_dict(orient='index'))
        
        if "매도" not in data_collection[company_name] or target_date not in data_collection[company_name]["매도"]:
            sell_value = stock.get_market_trading_value_by_date(target_date, target_date, ticker, on='매도')
            sell_value.index = sell_value.index.astype(str)
            
            if "매도" not in data_collection[company_name]:
                data_collection[company_name]["매도"] = {}
            
            data_collection[company_name]["매도"].update(sell_value.to_dict(orient='index'))
        
        if "일자별 거래실적 추이(거래량)" not in data_collection[company_name] or target_date not in data_collection[company_name]["일자별 거래실적 추이(거래량)"]:
            trading_volume = stock.get_market_trading_volume_by_date(target_date, target_date, ticker)
            trading_volume.index = trading_volume.index.astype(str)
            
            if "일자별 거래실적 추이(거래량)" not in data_collection[company_name]:
                data_collection[company_name]["일자별 거래실적 추이(거래량)"] = {}
            
            data_collection[company_name]["일자별 거래실적 추이(거래량)"].update(trading_volume.to_dict(orient='index'))
        
        if "투자자별 거래실적 추이(거래대금)" not in data_collection[company_name] or target_date not in data_collection[company_name]["투자자별 거래실적 추이(거래대금)"]:
            investor_value = stock.get_market_trading_value_by_investor(target_date, target_date, ticker)
            
            if "투자자별 거래실적 추이(거래대금)" not in data_collection[company_name]:
                data_collection[company_name]["투자자별 거래실적 추이(거래대금)"] = {}
            
            data_collection[company_name]["투자자별 거래실적 추이(거래대금)"].update(investor_value.to_dict(orient='index'))
        
        if "투자자별 거래실적 추이(거래량)" not in data_collection[company_name] or target_date not in data_collection[company_name]["투자자별 거래실적 추이(거래량)"]:
            investor_volume = stock.get_market_trading_volume_by_investor(target_date, target_date, ticker)
            
            if "투자자별 거래실적 추이(거래량)" not in data_collection[company_name]:
                data_collection[company_name]["투자자별 거래실적 추이(거래량)"] = {}
            
            data_collection[company_name]["투자자별 거래실적 추이(거래량)"].update(investor_volume.to_dict(orient='index'))
        
        if "종목별 시가총액 조회" not in data_collection[company_name] or target_date not in data_collection[company_name]["종목별 시가총액 조회"]:
            cap = stock.get_market_cap(target_date, target_date, ticker)
            cap.index = cap.index.astype(str)
            
            if "종목별 시가총액 조회" not in data_collection[company_name]:
                data_collection[company_name]["종목별 시가총액 조회"] = {}
            
            data_collection[company_name]["종목별 시가총액 조회"].update(cap.to_dict(orient='index'))
        
        if "종목별 공매도 현황" not in data_collection[company_name] or target_date not in data_collection[company_name]["종목별 공매도 현황"]:
            shorting_status = stock.get_shorting_status_by_date(target_date, target_date, ticker)
            shorting_status.index = shorting_status.index.astype(str)
            
            if "종목별 공매도 현황" not in data_collection[company_name]:
                data_collection[company_name]["종목별 공매도 현황"] = {}
            
            data_collection[company_name]["종목별 공매도 현황"].update(shorting_status.to_dict(orient='index'))
        
    # JSON 파일에 다시 저장
    with open(json_filename, 'w', encoding='utf-8') as file:
        json.dump(data_collection, file, ensure_ascii=False, indent=4)


# 원하는 날짜 설정
target_date = "20230825"
add_data_to_json(target_date, stocks, stocks_name)
